In [11]:
# import cv2 
import argparse

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

import datasets
import util.misc as utils
from models import build_model, build_model_hierINTR
import datasets.transforms as T
from datasets import build_dataset

In [14]:
def get_args_parser():
    parser = argparse.ArgumentParser('Set transformer detector', add_help=False)
    parser.add_argument('--lr_backbone', default=1.00e-5, type=float)
    parser.add_argument('--batch_size', default=1, type=int, choices=[1])

    # * Backbone
    parser.add_argument('--backbone', default='resnet50', type=str,
                        help="Name of the convolutional backbone to use")
    parser.add_argument('--dilation', action='store_true',
                        help="If true, we replace stride with dilation in the last convolutional block (DC5)")
    parser.add_argument('--position_embedding', default='sine', type=str, choices=('sine', 'learned'),
                        help="Type of positional embedding to use on top of the image features")
    parser.add_argument('--resume', default='path/to/intr_checkpoint_cub_detr_r50.pth',
                        help='resume from checkpoint')
    
    # * Transformer
    parser.add_argument('--enc_layers', default=6, type=int,
                        help="Number of encoding layers in the transformer")
    parser.add_argument('--dec_layers', default=6, type=int,
                        help="Number of decoding layers in the transformer")
    parser.add_argument('--dim_feedforward', default=2048, type=int,
                        help="Intermediate size of the feedforward layers in the transformer blocks")
    parser.add_argument('--hidden_dim', default=256, type=int,
                        help="Size of the embeddings (dimension of the transformer)")
    parser.add_argument('--dropout', default=0.1, type=float,
                        help="Dropout applied in the transformer") #default=0.1
    parser.add_argument('--nheads', default=8, type=int,
                        help="Number of attention heads inside the transformer's attentions")
    parser.add_argument('--num_queries', default=200, type=int,
                        help="Number of query slots")
    parser.add_argument('--pre_norm', action='store_true')

    # # * Dataset parameters
    parser.add_argument('--dataset_name', default='cub') 
    parser.add_argument('--dataset_path', default='/path/to/datasets', type=str) 
    parser.add_argument('--test', default="val", type=str, choices=["val", "test"])

    # * Device parameters
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')

    # Phylogeny parameters
    parser.add_argument("--phylo_config", type=str, default=None, 
                        help='path to the yaml file containing "phylogeny_path" and "phyloDistances_string"') # "./configs/cub27_phylogeny.yaml"
    
    return parser

In [4]:
data_mean_std = {
    "cub": (
        torch.tensor([0.485, 0.456, 0.406]),
        torch.tensor([0.229, 0.224, 0.225]),
    ),
}

class SingleImageDataset:
    def __init__(self, image_path, transform=None):
        self.image_path = image_path
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(self.image_path).convert('RGB')

        target = {}
        target["file_name"] = [str(self.image_path)]
        target["image_label"] = torch.tensor([0000], dtype=torch.int64)  # Set the target label as needed

        if self.transform is not None:
            img, target = self.transform(img, target)

        return img, target

    def __len__(self):
        return 1  

def make_transforms():
    mean, std = data_mean_std["cub"]

    normalize = T.Compose([
        T.ToTensor(),
        T.Normalize(mean, std)
    ])
    return T.Compose([
        T.RandomResize([800], max_size=1333),
        normalize,
    ])

def build_single_image_dataset(image_path):
    transform = make_transforms()
    dataset = SingleImageDataset(image_path=image_path, transform=transform)
    return dataset

def image_preprocessing(image_path):
    single_image_dataset = build_single_image_dataset(image_path)
    sampler_val = torch.utils.data.SequentialSampler(single_image_dataset)
    data_loader_val = DataLoader(single_image_dataset, args.batch_size, sampler=sampler_val,
                                 drop_last=False, collate_fn=utils.collate_fn, num_workers=1) 
    return data_loader_val

In [5]:
def load_model(args, model):
    checkpoint = torch.load(args.resume, map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    return model

In [6]:
@torch.no_grad()
def predicted_class(output, topk=(1,)):
    query_logits = output['query_logits']
    batch_size = query_logits.size(0)
    
    maxk = min(max(topk), query_logits.size(1))
    _, pred = query_logits.topk(maxk, dim=1, largest=True, sorted=True)
    return pred

In [7]:
def get_class_name(number, filename="demo_image/classes.txt"):
    with open(filename, "r") as file:
        lines = file.readlines()
    for line in lines:
        parts = line.strip().split(" ", 1)
        if parts[0] == str(number):
            class_name = parts[1]
            break
    else:
        return f"No class found for number {number}"

    class_name_parts = class_name.split(".")
    modified_name = " ".join(part.capitalize() for part in class_name_parts[1].split("_"))

    return modified_name


In [8]:
def SuperImposeHeatmap(attention, input_image):
    alpha=0.5
    avg_heatmap_resized = cv2.resize(attention, (input_image.shape[1], input_image.shape[0]), interpolation=cv2.INTER_CUBIC)
    avg_normalized_heatmap = (avg_heatmap_resized - np.min(avg_heatmap_resized)) / (np.max(avg_heatmap_resized) - np.min(avg_heatmap_resized))
    heatmap = (avg_normalized_heatmap * 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.medianBlur(heatmap,15)
    heatmap =  cv2.GaussianBlur(heatmap, (15, 15), 0)
    result = (input_image *alpha  + heatmap * (1-alpha)).astype(np.uint8)
    return result

# def get_heatmap(latent_activation, input_image, constant_color_scale=False):
#     image_a = latent_activation.cpu().numpy()
#     image_b = input_image.permute(1, 2, 0).cpu().numpy()
#     reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
#     if constant_color_scale:
#         reshaped_image_a = np.concatenate((reshaped_image_a, np.zeros((reshaped_image_a.shape[1], 1)), np.ones((reshaped_image_a.shape[1], 1))*255), axis=1)
    
#     normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
#     heatmap_colormap = plt.get_cmap('jet')
#     heatmap_colored = heatmap_colormap(normalized_heatmap)

#     if constant_color_scale:
#         heatmap_colored = heatmap_colored[:, :-2]
    
#     heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
#     image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
#     image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
#     result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
#     return np.array(result_image)

In [20]:


import pdb

def visualization(args, image_path, query_index, attention_scores, encoder_output):

    attention_score = attention_scores[5, :, :, :, :] # sixth layer of the decoder
    attention_score_pred_query = attention_score[:, :, query_index, :] # query corresponding to predicted class 

    # Load and resize the original image
    input_image = cv2.imread(image_path)
    input_image = cv2.resize(input_image, (0, 0), fx=0.8, fy=0.8)

    # Create a subplot grid for the original image and visualizations
    num_heads = attention_score_pred_query.shape[1] + 1  
    rows = 1
    cols = num_heads // rows

    # Plot the original image
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axes[0].imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    axes[0].axis('off')

    for head_index in range(attention_score_pred_query.shape[1]):
        heatmap_head = attention_score_pred_query[:, head_index, :].reshape(encoder_output.shape[2], encoder_output.shape[3]).detach().cpu().numpy()
        pdb.set_trace()
        result = SuperImposeHeatmap(heatmap_head, input_image) # heatmap_head-><class 'numpy.ndarray'>(25, 25) 

        if rows == 1 and cols == 1:
            ax = axes
        elif rows == 1:
            ax = axes[head_index + 1]
        elif cols == 1:
            ax = axes[head_index + 1]
        else:
            ax = axes[(head_index + 1) // cols, (head_index + 1) % cols]

        ax.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# def visualization(args, image_path, query_index, attention_scores, encoder_output):

#     attention_score = attention_scores[5, :, :, :, :] # sixth layer of the decoder
#     attention_score_pred_query = attention_score[:, :, query_index, :] # query corresponding to predicted class 

#     image = T.Resize(size=(args.image_size, args.image_size))(Image.open(image_path))
#     img_tensor = T.ToTensor()(image)

#     # Load and resize the original image
#     input_image = cv2.imread(image_path)
#     input_image = cv2.resize(input_image, (0, 0), fx=0.8, fy=0.8)

#     # Create a subplot grid for the original image and visualizations
#     num_heads = attention_score_pred_query.shape[1] + 1  
#     rows = 1
#     cols = num_heads // rows

#     # Plot the original image
#     fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
#     axes[0].imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
#     axes[0].axis('off')

#     for head_index in range(attention_score_pred_query.shape[1]):
#         heatmap_head = attention_score_pred_query[:, head_index, :].reshape(encoder_output.shape[2], encoder_output.shape[3]).detach().cpu().numpy()
#         pdb.set_trace()
#         result = SuperImposeHeatmap(heatmap_head, input_image) # heatmap_head-><class 'numpy.ndarray'>(25, 25) 

#         if rows == 1 and cols == 1:
#             ax = axes
#         elif rows == 1:
#             ax = axes[head_index + 1]
#         elif cols == 1:
#             ax = axes[head_index + 1]
#         else:
#             ax = axes[(head_index + 1) // cols, (head_index + 1) % cols]

#         ax.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
#         ax.axis('off')

#     plt.tight_layout()
#     plt.show()

In [23]:
import pdb

def main(args):

    dataset_val = build_dataset(image_set=args.test, args=args)
    label_to_spcname = {label: classname for classname, label in dataset_val.class_to_idx.items()}
    
    image_path = "/home/harishbabu/data/cub190_imgnet/train/cub_012_Yellow_headed_Blackbird/Yellow_Headed_Blackbird_0017_8511.jpg"
    
    test_image_loader=image_preprocessing(image_path)

    device = torch.device(args.device)
    # model, _ = build_model(args)
    model, criterion, spclabel_to_anclabels= build_model_hierINTR(args, label_to_spcname)
    model.to(device)
    model=load_model(args, model)
    model.eval()

    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'
    for samples, targets in metric_logger.log_every(test_image_loader, 1, header):
        samples = samples.to(device)
    
        outputs, encoder_output, _, attention_scores, avg_attention_scores = model(samples)

        pdb.set_trace()
        pred_class_index=predicted_class(outputs, topk=(1, 1))
        # print ("Species predicted by INTR is:",get_class_name(int(pred_class_index[0][0])+1), "\n")
        print ("Species predicted by HierINTR is:",label_to_spcname[int(pred_class_index[0][0])], "\n")

        visualization(args, image_path, int(pred_class_index[0][0]), attention_scores, encoder_output)

if __name__ == '__main__':
    parser = argparse.ArgumentParser('INTR interpretation visualization script', parents=[get_args_parser()])
    args = parser.parse_args(args=[])

    args.phylo_config = 'configs/cub190_phylogeny_disc4.yaml'
    args.resume = 'output_HierINTR/cub190_imgnet/001_cub190-imgnet_disc=4/cub_checkpoint.pth'
    args.dataset_path = "/home/harishbabu/data/"
    args.dataset_name = "cub190_imgnet"
    main(args)

In build function
Built backbone
Built transformer
Built tree
Set anc labels
> /tmp/ipykernel_317451/1458321408.py(27)main()
     25 
     26         pdb.set_trace()
---> 27         pred_class_index=predicted_class(outputs, topk=(1, 1))
     28         # print ("Species predicted by INTR is:",get_class_name(int(pred_class_index[0][0])+1), "\n")
     29         print ("Species predicted by HierINTR is:",label_to_spcname[int(pred_class_index[0][0])], "\n")



ipdb>  type(attention_scores)


<class 'torch.Tensor'>


ipdb>  attention_scores.shape


torch.Size([6, 1, 8, 190, 625])


...

ipdb>  head_index = 0
ipdb>  heatmap_head = attention_score_pred_query[:, head_index, :].reshape(encoder_output.shape[2], encoder_output.shape[3]).detach().cpu().numpy()
ipdb>  heatmap_head.shape


(25, 25)


ipdb>  type(heatmap_head)


<class 'numpy.ndarray'>


ipdb>  q


In [17]:
!pwd

/home/harishbabu/projects/INTR
